In [1]:
import tensorflow as tf
import numpy as np
import os
import cv2
from tqdm import tqdm

In [2]:
from random import shuffle

In [3]:
train_data = np.load('/train/train_data.npy')

In [4]:
X_train = np.array([i[0] for i in train_data]).reshape(-1, 2500)
Y_train = np.array([i[1] for i in train_data])

In [5]:
X_train = X_train.astype(np.float32)
Y_train = Y_train.astype(np.float32)

In [6]:
def conv2d(x, W, b, strides = 1):
    x = tf.nn.conv2d(x, W, strides = [1, strides, strides, 1], padding = 'SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k = 2):
    return tf.nn.max_pool(x, ksize = [1, k, k, 1], strides = [1, k, k, 1], padding = 'SAME')

In [7]:
def conv_net(x, weights, biases, dropout):
      
    x = tf.reshape(x, shape=[-1, 50, 50, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)
    
    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
    conv3 = maxpool2d(conv3, k=2)
    
    conv4 = conv2d(conv3, weights['wc4'], biases['bc4'])
    conv4 = maxpool2d(conv4, k=2)
    
    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv4, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [8]:
learning_rate = 0.001
training_iters = 200000
batch_size = 100
display_step = 10

# Network Parameters
n_input = 2500 # MNIST data input (img shape: 28*28)
n_classes = 2 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

# tf Graph input
X = tf.placeholder(tf.float32, shape = [None, n_input], name = 'X')
Y = tf.placeholder(tf.float32, shape = [None, n_classes], name = 'Y')
keep_prob = tf.placeholder(tf.float32, name = 'dropout') #dropout (keep probability)

In [9]:
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wc3': tf.Variable(tf.random_normal([5, 5, 64, 32])),
    'wc4': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    
    'wd1': tf.Variable(tf.random_normal([4*4*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bc3': tf.Variable(tf.random_normal([32])),
    'bc4': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = conv_net(X, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

In [20]:
with tf.Session() as sess:
    sess.run(init)
    step = 1
    while step * batch_size < training_iters:
        
        batch_x, batch_y = next_batch(batch_size, step - 1)
        sess.run(optimizer, feed_dict ={X: batch_x, Y: batch_y, keep_prob: dropout})
        
        if step % display_step == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict = {X: batch_x, Y: batch_y, keep_prob: 1.})
            
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    
    print("Optimization Finished!")

Iter 1000, Minibatch Loss= 578906048.000000, Training Accuracy= 0.50000
Iter 2000, Minibatch Loss= 452219776.000000, Training Accuracy= 0.59000
Iter 3000, Minibatch Loss= 504508544.000000, Training Accuracy= 0.57000
Iter 4000, Minibatch Loss= 286477920.000000, Training Accuracy= 0.53000
Iter 5000, Minibatch Loss= 293801664.000000, Training Accuracy= 0.53000
Iter 6000, Minibatch Loss= 144225120.000000, Training Accuracy= 0.60000
Iter 7000, Minibatch Loss= 160042736.000000, Training Accuracy= 0.66000
Iter 8000, Minibatch Loss= 171133168.000000, Training Accuracy= 0.53000
Iter 9000, Minibatch Loss= 127253448.000000, Training Accuracy= 0.60000
Iter 10000, Minibatch Loss= 150461536.000000, Training Accuracy= 0.58000
Iter 11000, Minibatch Loss= 130674288.000000, Training Accuracy= 0.60000
Iter 12000, Minibatch Loss= 121167360.000000, Training Accuracy= 0.62000
Iter 13000, Minibatch Loss= 120932608.000000, Training Accuracy= 0.55000
Iter 14000, Minibatch Loss= 138566960.000000, Training Accur

Iter 116000, Minibatch Loss= 838410.375000, Training Accuracy= 0.49000
Iter 117000, Minibatch Loss= 766181.875000, Training Accuracy= 0.50000
Iter 118000, Minibatch Loss= 323360.125000, Training Accuracy= 0.48000
Iter 119000, Minibatch Loss= 587101.750000, Training Accuracy= 0.58000
Iter 120000, Minibatch Loss= 1210474.875000, Training Accuracy= 0.47000
Iter 121000, Minibatch Loss= 1026712.062500, Training Accuracy= 0.56000
Iter 122000, Minibatch Loss= 697859.750000, Training Accuracy= 0.49000
Iter 123000, Minibatch Loss= 545647.687500, Training Accuracy= 0.55000
Iter 124000, Minibatch Loss= 507195.031250, Training Accuracy= 0.43000
Iter 125000, Minibatch Loss= 432602.656250, Training Accuracy= 0.59000
Iter 126000, Minibatch Loss= 97750.164062, Training Accuracy= 0.60000
Iter 127000, Minibatch Loss= 509658.906250, Training Accuracy= 0.55000
Iter 128000, Minibatch Loss= 1529227.500000, Training Accuracy= 0.46000
Iter 129000, Minibatch Loss= 175068.640625, Training Accuracy= 0.61000
Iter

In [19]:
idxs = np.random.permutation(range(len(X_train)))

def next_batch(size, num): 
    global idxs
    num = num % 250
    if num == 0:
        idxs = np.random.permutation(range(len(X_train)))
    idxs_i = idxs[num * size : (num + 1) * size]
    return X_train[idxs_i], Y_train[idxs_i]

In [14]:
a, b = next_batch(100, 250)
c, d = next_batch(100,1)

In [17]:
assert(a.all() == c.all())

In [18]:
3 % 2

1